In [13]:
import numpy as np
import os
import sys

import tensorflow as tf
from tensorflow.python.ops import array_ops, math_ops      #for math operations division_no_nan
from tensorflow.keras.layers import *
from sklearn.metrics import roc_curve, auc
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

import hls4ml
from hls4ml.model.profiling import numerical, activations_keras, boxplot

sys.path.append('/usersc/bz18310/previous_notebook/cms-l1-triggers')

from utils.analysis import eff_rate, optimal_eff_rate
from utils.preprocessing import resize
from utils.plotting import *
from utils.hls4ml_helpers import *

## Notes

1) Number of parameters increases when second conv layer removed

In [14]:
def make_model(in_shape, second_layer=True):

    model = tf.keras.Sequential()

    model.add(Conv2D(4, kernel_size=(3,3), input_shape=in_shape))
    model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    if second_layer:
        model.add(Conv2D(8, kernel_size=(3,3)))
        model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
        model.add(BatchNormalization(axis=1))
        model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(24))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))

    model.add(Dense(1))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('sigmoid'))

    model.build(input_shape=in_shape)

    model.summary()
    
    model = compiler(model)
    
    return model

def compiler(model_name):
    opt = tf.keras.optimizers.Adam(0.001)
    sensitivity_metric = tf.keras.metrics.SensitivityAtSpecificity(name='sens_at_spec',
                                                                             specificity=0.99925,     
                                                                             num_thresholds=20000)     
    auc_metric = tf.keras.metrics.AUC(name='auc', num_thresholds=200)   
    metrics = ['accuracy', sensitivity_metric, auc_metric]

    model_name.compile(optimizer=opt, loss='binary_crossentropy', metrics=metrics)
    
    return model_name

def trainer(model_name, train_X, train_y):     

    model_name.fit(train_X, 
               train_y, 
               epochs=50, 
               verbose=1,
               batch_size=512, 
               validation_split=.2,   
               shuffle=True,
               callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                             patience=5,
                                                             restore_best_weights=True),     
                            pruning_callbacks.UpdatePruningStep()])

In [15]:
#coarse model data 20x12
data = np.load('./data_hh4b_20x12_160000.npz')
train_X_c = data['train_X']      #data for training the quantized model
train_y_c = data['train_y']      #data labels
test_X_c = data['test_X']
test_y_c = data['test_y']
data = 0

In [16]:
#120x12 fine model data
train_X = np.load('/storage1/bz18310/hist_data/train_test_120x72/train_X.npy')
train_y = np.load('/storage1/bz18310/hist_data/train_test_120x72/train_y.npy')
test_X = np.load('/storage1/bz18310/hist_data/train_test_120x72/test_X.npy')
test_y = np.load('/storage1/bz18310/hist_data/train_test_120x72/test_y.npy')

In [19]:
model_120_full = make_model((120,72,1), second_layer=True)
model_120_half = make_model((120,72,1), second_layer=False)
model_20_full = make_model((20,12,1), second_layer=True)
model_20_half = make_model((20,12,1), second_layer=False)   #note the number parameters increases here

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 118, 70, 4)        40        
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 59, 35, 4)         0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 59, 35, 4)         236       
_________________________________________________________________
activation_23 (Activation)   (None, 59, 35, 4)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 57, 33, 8)         296       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 28, 16, 8)         0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 28, 16, 8)        

In [20]:
trainer(model_120_full, train_X, train_y)
trainer(model_120_half, train_X, train_y)
trainer(model_20_full, train_X_c, train_y_c)
trainer(model_20_half, train_X_c, train_y_c)

Epoch 1/50
442/442 [==============================] - 158s 355ms/step - loss: 0.3528 - accuracy: 0.9120 - sens_at_spec: 0.2088 - auc: 0.9922 - val_loss: 0.2980 - val_accuracy: 0.9638 - val_sens_at_spec: 0.3826 - val_auc: 0.9955
Epoch 2/50
442/442 [==============================] - 154s 349ms/step - loss: 0.2120 - accuracy: 0.9765 - sens_at_spec: 0.1089 - auc: 0.9949 - val_loss: 0.1745 - val_accuracy: 0.9784 - val_sens_at_spec: 0.0856 - val_auc: 0.9954
Epoch 3/50
442/442 [==============================] - 154s 349ms/step - loss: 0.1516 - accuracy: 0.9806 - sens_at_spec: 0.1926 - auc: 0.9955 - val_loss: 0.1326 - val_accuracy: 0.9799 - val_sens_at_spec: 0.0603 - val_auc: 0.9956
Epoch 4/50
442/442 [==============================] - 155s 351ms/step - loss: 0.1176 - accuracy: 0.9821 - sens_at_spec: 0.2755 - auc: 0.9962 - val_loss: 0.1071 - val_accuracy: 0.9817 - val_sens_at_spec: 0.0803 - val_auc: 0.9962
Epoch 5/50
442/442 [==============================] - 155s 352ms/step - loss: 0.0959 - a

Epoch 16/50
442/442 [==============================] - 121s 275ms/step - loss: 0.0317 - accuracy: 0.9904 - sens_at_spec: 0.9587 - auc: 0.9991 - val_loss: 0.0648 - val_accuracy: 0.9798 - val_sens_at_spec: 0.0541 - val_auc: 0.9952
Epoch 1/50
188/188 [==============================] - 20s 63ms/step - loss: 0.3670 - accuracy: 0.9403 - sens_at_spec: 0.2998 - auc: 0.9872 - val_loss: 0.8925 - val_accuracy: 0.5021 - val_sens_at_spec: 0.5102 - val_auc: 0.9492
Epoch 2/50
188/188 [==============================] - 11s 58ms/step - loss: 0.2817 - accuracy: 0.9676 - sens_at_spec: 0.1916 - auc: 0.9929 - val_loss: 0.8395 - val_accuracy: 0.5340 - val_sens_at_spec: 0.4646 - val_auc: 0.9500
Epoch 3/50
188/188 [==============================] - 11s 56ms/step - loss: 0.2457 - accuracy: 0.9688 - sens_at_spec: 0.1782 - auc: 0.9926 - val_loss: 0.3210 - val_accuracy: 0.8923 - val_sens_at_spec: 0.5538 - val_auc: 0.9937
Epoch 4/50
188/188 [==============================] - 11s 58ms/step - loss: 0.2193 - accuracy

188/188 [==============================] - 10s 54ms/step - loss: 0.1770 - accuracy: 0.9751 - sens_at_spec: 0.3592 - auc: 0.9955 - val_loss: 0.1593 - val_accuracy: 0.9763 - val_sens_at_spec: 0.4301 - val_auc: 0.9963
Epoch 7/50
188/188 [==============================] - 10s 53ms/step - loss: 0.1620 - accuracy: 0.9755 - sens_at_spec: 0.3613 - auc: 0.9955 - val_loss: 0.1457 - val_accuracy: 0.9753 - val_sens_at_spec: 0.5682 - val_auc: 0.9964
Epoch 8/50
188/188 [==============================] - 10s 54ms/step - loss: 0.1496 - accuracy: 0.9752 - sens_at_spec: 0.3558 - auc: 0.9957 - val_loss: 0.1403 - val_accuracy: 0.9768 - val_sens_at_spec: 0.5367 - val_auc: 0.9965
Epoch 9/50
188/188 [==============================] - 10s 55ms/step - loss: 0.1390 - accuracy: 0.9759 - sens_at_spec: 0.4812 - auc: 0.9959 - val_loss: 0.1590 - val_accuracy: 0.9751 - val_sens_at_spec: 0.5793 - val_auc: 0.9965
Epoch 10/50
188/188 [==============================] - 10s 52ms/step - loss: 0.1306 - accuracy: 0.9753 - se

188/188 [==============================] - 10s 54ms/step - loss: 0.0619 - accuracy: 0.9790 - sens_at_spec: 0.7909 - auc: 0.9972 - val_loss: 0.0703 - val_accuracy: 0.9767 - val_sens_at_spec: 0.5968 - val_auc: 0.9963
Epoch 43/50
188/188 [==============================] - 10s 51ms/step - loss: 0.0619 - accuracy: 0.9792 - sens_at_spec: 0.7944 - auc: 0.9972 - val_loss: 0.0682 - val_accuracy: 0.9774 - val_sens_at_spec: 0.6479 - val_auc: 0.9964
Epoch 44/50
188/188 [==============================] - 10s 54ms/step - loss: 0.0613 - accuracy: 0.9791 - sens_at_spec: 0.7704 - auc: 0.9973 - val_loss: 0.0730 - val_accuracy: 0.9756 - val_sens_at_spec: 0.5834 - val_auc: 0.9962
Epoch 45/50
188/188 [==============================] - 10s 53ms/step - loss: 0.0614 - accuracy: 0.9790 - sens_at_spec: 0.7751 - auc: 0.9972 - val_loss: 0.0677 - val_accuracy: 0.9777 - val_sens_at_spec: 0.6230 - val_auc: 0.9965
Epoch 46/50
188/188 [==============================] - 10s 55ms/step - loss: 0.0609 - accuracy: 0.9792 -

In [9]:
push = lambda x: x > .5       #function to check if number greater than .5

def hls_model_accuracy(hls_model, test_X, test_y):                              #function for testing accurcry of hls model
    return accuracy_score(test_y, [push(i) for i in hls_model.predict(test_X)])

def get_timing(build_result):
    return (build_result['BestLatency'], build_result['WorstLatency'],
            build_result['IntervalMin'], build_result['IntervalMax'])


class CustomSensitivityAtSpecificity(tf.keras.metrics.SensitivityAtSpecificity):     #specificity TN/(TN+FP) 
                                                                                
    def __init__(self, specificity, num_thresholds=200, name=None, dtype=None):
        if specificity < 0 or specificity > 1:
            raise ValueError('`specificity` must be in the range [0, 1].')
        self.specificity = specificity
        self.num_thresholds = num_thresholds
        super().__init__(
            specificity, num_thresholds=num_thresholds, name=name, dtype=dtype)  #from the tf.kera.metrics.SensitivityAtSpecificity
                                                                                 #class, creates out object
    def result(self):
        specificities = math_ops.div_no_nan(
        self.true_negatives, self.true_negatives + self.false_positives)   #from keras class super()
        sensitivities = math_ops.div_no_nan(
        self.true_positives, self.true_positives + self.false_negatives)   
        return self._find_max_under_constraint(
                    specificities, sensitivities, math_ops.greater_equal)  #What?

    def get_threshold(self):
        specificities = math_ops.div_no_nan(
        self.true_negatives, self.true_negatives + self.false_positives)
        
        sensitivities = math_ops.div_no_nan(
        self.true_positives, self.true_positives + self.false_negatives)    #Sensitivity TP/(TP+FN)
        
        specs_above_thresh = array_ops.where(math_ops.greater_equal(specificities, self.value))   #What?
        
        return math_ops.reduce_min(array_ops.gather(self.thresholds, specs_above_thresh)).numpy()
       